In [1]:
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [39]:
df = pd.read_csv('1617_AllGames_byTeam_FF.csv',index_col=0)
df.head()

,TEAM,MATCH UP,GAME DATE,MIN,EFG%,FTA RATE,TOV%,TOV,OREB%,OPP_EFG%,...,OPP_TOV,OPP_OREB%,OPP,Home/Away,REST,PTS,OPP_PTS,+/-,WIN,OPP_REST
0,POR,POR vs. UTA,10/25/16,48,0.607,0.293,0.140,14.0,0.167,0.537,...,14.4,0.171,UTA,1,0,113,104,9,1,0
1,UTA,UTA @ POR,10/25/16,48,0.537,0.195,0.144,14.4,0.171,0.607,...,14.0,0.167,POR,0,0,104,113,-9,0,0
2,NYK,NYK @ CLE,10/25/16,48,0.420,0.230,0.179,17.9,0.245,0.548,...,14.1,0.275,CLE,0,0,88,117,-29,0,0
3,GSW,GSW vs. SAS,10/25/16,48,0.512,0.212,0.159,15.9,0.190,0.541,...,13.7,0.438,SAS,1,0,100,129,-29,0,0
4,SAS,SAS @ GSW,10/25/16,48,0.541,0.265,0.137,13.7,0.438,0.512,...,15.9,0.190,GSW,0,0,129,100,29,1,0


In [40]:
del df['MATCH UP']
del df['TOV']
del df['OPP_TOV']

df.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,PTS,OPP_PTS,+/-,WIN,OPP_REST
0,POR,10/25/16,48,0.607,0.293,0.140,0.167,0.537,0.195,0.144,0.171,UTA,1,0,113,104,9,1,0
1,UTA,10/25/16,48,0.537,0.195,0.144,0.171,0.607,0.293,0.140,0.167,POR,0,0,104,113,-9,0,0
2,NYK,10/25/16,48,0.420,0.230,0.179,0.245,0.548,0.202,0.141,0.275,CLE,0,0,88,117,-29,0,0
3,GSW,10/25/16,48,0.512,0.212,0.159,0.190,0.541,0.265,0.137,0.438,SAS,1,0,100,129,-29,0,0
4,SAS,10/25/16,48,0.541,0.265,0.137,0.438,0.512,0.212,0.159,0.190,GSW,0,0,129,100,29,1,0


In [42]:
### df_all preserves all data
df_all = df
df_all.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,PTS,OPP_PTS,+/-,WIN,OPP_REST
0,POR,10/25/16,48,0.607,0.293,0.140,0.167,0.537,0.195,0.144,0.171,UTA,1,0,113,104,9,1,0
1,UTA,10/25/16,48,0.537,0.195,0.144,0.171,0.607,0.293,0.140,0.167,POR,0,0,104,113,-9,0,0
2,NYK,10/25/16,48,0.420,0.230,0.179,0.245,0.548,0.202,0.141,0.275,CLE,0,0,88,117,-29,0,0
3,GSW,10/25/16,48,0.512,0.212,0.159,0.190,0.541,0.265,0.137,0.438,SAS,1,0,100,129,-29,0,0
4,SAS,10/25/16,48,0.541,0.265,0.137,0.438,0.512,0.212,0.159,0.190,GSW,0,0,129,100,29,1,0


### predict Win

#### without one hot encoding--delete all teams

In [43]:
#del df['TEAM']
#del df['OPP']
del df['PTS']
del df['OPP_PTS']
del df['+/-']
df['W/L']=df['WIN']
del df['WIN']
df.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,POR,10/25/16,48,0.607,0.293,0.140,0.167,0.537,0.195,0.144,0.171,UTA,1,0,0,1
1,UTA,10/25/16,48,0.537,0.195,0.144,0.171,0.607,0.293,0.140,0.167,POR,0,0,0,0
2,NYK,10/25/16,48,0.420,0.230,0.179,0.245,0.548,0.202,0.141,0.275,CLE,0,0,0,0
3,GSW,10/25/16,48,0.512,0.212,0.159,0.190,0.541,0.265,0.137,0.438,SAS,1,0,0,0
4,SAS,10/25/16,48,0.541,0.265,0.137,0.438,0.512,0.212,0.159,0.190,GSW,0,0,0,1


### df_test: DataFrame stores all rows for test, for each team, 8 factors averaged from its previous 5 games 

In [78]:
df_test=df
len(df_test)

2460

In [79]:
list_teams = ['ATL','BKN','BOS','CHA','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM','MIA',
              'MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHX','POR','SAC','SAS','TOR','UTA','WAS']
df_allTeams = pd.DataFrame()    
for i in list_teams:
    df_allTeams = pd.concat([df_allTeams, df_test.loc[df['TEAM'] == i]])
df_allTeams.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
33,ATL,10/27/16,48,0.568,0.205,0.204,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
66,ATL,10/29/16,48,0.524,0.271,0.119,0.146,0.381,0.188,0.195,0.089,PHI,0,2,3,1
82,ATL,10/31/16,48,0.487,0.595,0.136,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
109,ATL,11/2/16,48,0.547,0.341,0.170,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
154,ATL,11/4/16,48,0.416,0.434,0.168,0.222,0.419,0.438,0.184,0.229,WAS,0,2,2,0


### set all MIN to 48

In [87]:
df_allTeams.loc[:,'MIN'] = 48
df_allTeams.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
33,ATL,10/27/16,48,0.568,0.205,0.204,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
66,ATL,10/29/16,48,0.524,0.271,0.119,0.146,0.381,0.188,0.195,0.089,PHI,0,2,3,1
82,ATL,10/31/16,48,0.487,0.595,0.136,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
109,ATL,11/2/16,48,0.547,0.341,0.170,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
154,ATL,11/4/16,48,0.416,0.434,0.168,0.222,0.419,0.438,0.184,0.229,WAS,0,2,2,0
163,ATL,11/5/16,48,0.576,0.282,0.195,0.295,0.506,0.153,0.248,0.279,HOU,1,1,3,1
200,ATL,11/8/16,48,0.574,0.296,0.191,0.297,0.429,0.253,0.058,0.250,CLE,0,3,3,1
213,ATL,11/9/16,48,0.560,0.325,0.219,0.361,0.500,0.231,0.132,0.186,CHI,1,1,2,1
272,ATL,11/12/16,48,0.574,0.147,0.157,0.341,0.500,0.241,0.188,0.213,PHI,1,3,1,1
305,ATL,11/15/16,48,0.500,0.238,0.203,0.162,0.444,0.157,0.227,0.370,MIA,0,3,1,1


In [89]:
### saving byTeam
df_allTeams.to_csv('AllGames_byTeam.csv', index = False)
df_allTeams = pd.read_csv('AllGames_byTeam.csv', encoding = 'latin1')
df_allTeams.to_csv('AllGames_byTeam.csv')

In [127]:
df_allTeams = pd.read_csv('AllGames_byTeam.csv', encoding = 'latin1',index_col=0)
df_allTeams.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.568,0.205,0.204,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
1,ATL,10/29/16,48,0.524,0.271,0.119,0.146,0.381,0.188,0.195,0.089,PHI,0,2,3,1
2,ATL,10/31/16,48,0.487,0.595,0.136,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
3,ATL,11/2/16,48,0.547,0.341,0.170,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
4,ATL,11/4/16,48,0.416,0.434,0.168,0.222,0.419,0.438,0.184,0.229,WAS,0,2,2,0


In [114]:
list_efg=df_allTeams.loc[:,'OREB%']
#list_efg

In [115]:
list_5efg=[] #avg of 5 as list
for i in range(30):
    x=((i+1)*82)
    s=(i*82)
    list_5efg.append((list_efg[x-1]+list_efg[x-2]+list_efg[x-3]+list_efg[x-4]+list_efg[x-5])/5)
    list_5efg.append((list_efg[x-1]+list_efg[x-2]+list_efg[x-3]+list_efg[x-4]+list_efg[s])/5)
    list_5efg.append((list_efg[x-1]+list_efg[x-2]+list_efg[x-3]+list_efg[s]+list_efg[s+1])/5)
    list_5efg.append((list_efg[x-1]+list_efg[x-2]+list_efg[s]+list_efg[s+1]+list_efg[s+2])/5)
    list_5efg.append((list_efg[x-1]+list_efg[s]+list_efg[s+1]+list_efg[s+2]+list_efg[s+3])/5)
    #list_5efg.append((i+1)*82)
    for j in range(i*82,(i+1)*82-5):
        list_5efg.append((list_efg[j]+list_efg[j+1]+list_efg[j+2]+list_efg[j+3]+list_efg[j+4])/5)
        

In [103]:
(0.568+0.524+0.487+0.547+0.416)/5

0.5084000000000001

In [107]:
###DO NOT RUN THIS
list_avg_efg = list_5efg
list_avg_efg

[0.54480000000000006,
 0.54720000000000002,
 0.53280000000000005,
 0.52700000000000002,
 0.52380000000000004,
 0.50840000000000007,
 0.51000000000000001,
 0.52000000000000002,
 0.53459999999999996,
 0.54000000000000004,
 0.55679999999999996,
 0.55279999999999996,
 0.5391999999999999,
 0.51419999999999999,
 0.503,
 0.49359999999999998,
 0.45459999999999995,
 0.4556,
 0.46159999999999995,
 0.46920000000000001,
 0.4582,
 0.47359999999999997,
 0.46879999999999999,
 0.47800000000000004,
 0.47640000000000005,
 0.51019999999999999,
 0.54239999999999999,
 0.54959999999999998,
 0.56880000000000008,
 0.55059999999999998,
 0.53259999999999996,
 0.50679999999999992,
 0.4824,
 0.45879999999999999,
 0.47400000000000003,
 0.49959999999999993,
 0.5,
 0.51579999999999993,
 0.5212,
 0.51340000000000008,
 0.51360000000000006,
 0.52500000000000002,
 0.51360000000000006,
 0.51639999999999997,
 0.5242,
 0.49739999999999995,
 0.51559999999999995,
 0.51419999999999999,
 0.504,
 0.48460000000000003,
 0.5031999

In [128]:
for i in range(2460):
    df_allTeams.loc[i,'EFG%'] = list_avg_efg[i]

df_allTeams.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.5448,0.205,0.204,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
1,ATL,10/29/16,48,0.5472,0.271,0.119,0.146,0.381,0.188,0.195,0.089,PHI,0,2,3,1
2,ATL,10/31/16,48,0.5328,0.595,0.136,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
3,ATL,11/2/16,48,0.5270,0.341,0.170,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
4,ATL,11/4/16,48,0.5238,0.434,0.168,0.222,0.419,0.438,0.184,0.229,WAS,0,2,2,0


In [110]:
list_avg_fta = list_5efg
list_avg_fta

[0.27159999999999995,
 0.23619999999999997,
 0.27240000000000003,
 0.31259999999999999,
 0.3352,
 0.36919999999999997,
 0.3846,
 0.3896,
 0.33560000000000001,
 0.29680000000000001,
 0.2576,
 0.28739999999999999,
 0.26860000000000001,
 0.27179999999999999,
 0.26600000000000001,
 0.2702,
 0.21299999999999999,
 0.19539999999999999,
 0.1978,
 0.1966,
 0.19019999999999998,
 0.193,
 0.25339999999999996,
 0.21680000000000002,
 0.26340000000000002,
 0.26519999999999999,
 0.30780000000000002,
 0.26180000000000003,
 0.29180000000000006,
 0.25460000000000005,
 0.28420000000000006,
 0.25080000000000002,
 0.26700000000000002,
 0.25980000000000003,
 0.28659999999999997,
 0.25419999999999998,
 0.251,
 0.28399999999999997,
 0.30980000000000002,
 0.30120000000000002,
 0.30180000000000001,
 0.31680000000000008,
 0.32520000000000004,
 0.28900000000000003,
 0.28959999999999997,
 0.30359999999999998,
 0.29660000000000003,
 0.26199999999999996,
 0.28280000000000005,
 0.28100000000000003,
 0.2762000000000000

In [129]:
for i in range(2460):
    df_allTeams.loc[i,'FTA RATE'] = list_avg_fta[i]

df_allTeams.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.5448,0.2716,0.204,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
1,ATL,10/29/16,48,0.5472,0.2362,0.119,0.146,0.381,0.188,0.195,0.089,PHI,0,2,3,1
2,ATL,10/31/16,48,0.5328,0.2724,0.136,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
3,ATL,11/2/16,48,0.5270,0.3126,0.170,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
4,ATL,11/4/16,48,0.5238,0.3352,0.168,0.222,0.419,0.438,0.184,0.229,WAS,0,2,2,0


In [113]:
list_avg_tov = list_5efg
list_avg_tov

[0.15540000000000001,
 0.1648,
 0.16599999999999998,
 0.17180000000000001,
 0.16920000000000002,
 0.15940000000000001,
 0.15760000000000002,
 0.17200000000000001,
 0.18859999999999999,
 0.186,
 0.193,
 0.18640000000000001,
 0.182,
 0.16500000000000001,
 0.16400000000000001,
 0.1676,
 0.16419999999999998,
 0.155,
 0.1646,
 0.17480000000000001,
 0.14500000000000002,
 0.154,
 0.16460000000000002,
 0.1578,
 0.15120000000000003,
 0.1668,
 0.15700000000000003,
 0.14680000000000001,
 0.14860000000000001,
 0.13679999999999998,
 0.12279999999999999,
 0.1356,
 0.1376,
 0.1356,
 0.13619999999999999,
 0.1482,
 0.13700000000000001,
 0.1318,
 0.12520000000000001,
 0.12820000000000001,
 0.14099999999999999,
 0.13139999999999999,
 0.13579999999999998,
 0.15279999999999999,
 0.15379999999999999,
 0.12899999999999998,
 0.13699999999999998,
 0.14859999999999998,
 0.12979999999999997,
 0.13419999999999999,
 0.15699999999999997,
 0.1638,
 0.15179999999999999,
 0.15279999999999999,
 0.156,
 0.13419999999999

In [130]:
for i in range(2460):
    df_allTeams.loc[i,'TOV%'] = list_avg_tov[i]

df_allTeams.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.5448,0.2716,0.1554,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
1,ATL,10/29/16,48,0.5472,0.2362,0.1648,0.146,0.381,0.188,0.195,0.089,PHI,0,2,3,1
2,ATL,10/31/16,48,0.5328,0.2724,0.1660,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
3,ATL,11/2/16,48,0.5270,0.3126,0.1718,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
4,ATL,11/4/16,48,0.5238,0.3352,0.1692,0.222,0.419,0.438,0.184,0.229,WAS,0,2,2,0


In [116]:
list_avg_oreb = list_5efg
list_avg_oreb

[0.23280000000000003,
 0.23600000000000004,
 0.23599999999999999,
 0.26539999999999997,
 0.253,
 0.25840000000000002,
 0.25080000000000002,
 0.28099999999999997,
 0.28379999999999994,
 0.30320000000000003,
 0.29120000000000001,
 0.24939999999999998,
 0.22639999999999999,
 0.20699999999999999,
 0.16799999999999998,
 0.20800000000000002,
 0.21379999999999999,
 0.20859999999999998,
 0.2112,
 0.22199999999999998,
 0.17399999999999999,
 0.20400000000000001,
 0.20619999999999999,
 0.22259999999999999,
 0.22440000000000002,
 0.27219999999999994,
 0.30500000000000005,
 0.31159999999999999,
 0.26919999999999999,
 0.26300000000000001,
 0.22840000000000002,
 0.21660000000000004,
 0.23519999999999999,
 0.26439999999999997,
 0.27359999999999995,
 0.25419999999999998,
 0.22919999999999999,
 0.19979999999999998,
 0.2024,
 0.20979999999999999,
 0.26719999999999999,
 0.253,
 0.25579999999999992,
 0.24580000000000002,
 0.24479999999999999,
 0.2056,
 0.20500000000000002,
 0.20699999999999999,
 0.2082,
 0

In [131]:
for i in range(2460):
    df_allTeams.loc[i,'OREB%'] = list_avg_oreb[i]

df_allTeams.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.5448,0.2716,0.1554,0.2328,0.457,0.191,0.174,0.240,WAS,1,0,0,1
1,ATL,10/29/16,48,0.5472,0.2362,0.1648,0.2360,0.381,0.188,0.195,0.089,PHI,0,2,3,1
2,ATL,10/31/16,48,0.5328,0.2724,0.1660,0.2360,0.547,0.253,0.197,0.188,SAC,1,2,2,1
3,ATL,11/2/16,48,0.5270,0.3126,0.1718,0.2654,0.583,0.222,0.146,0.289,LAL,1,2,1,0
4,ATL,11/4/16,48,0.5238,0.3352,0.1692,0.2530,0.419,0.438,0.184,0.229,WAS,0,2,2,0


In [132]:
list_opp_efg=[]
list_opp_fta=[]
list_opp_tov=[]
list_opp_oreb=[]
for i in range(2460):
    for j in range(2460):
        if df_allTeams.loc[j,'TEAM']==df_allTeams.loc[i,'OPP'] and df_allTeams.loc[j,'GAME DATE']==df_allTeams.loc[i,'GAME DATE']:
            print(i,df_allTeams.loc[i,'OPP'],df_allTeams.loc[j,'TEAM'])
            list_opp_efg.append(df_allTeams.loc[j,'EFG%'])
            list_opp_fta.append(df_allTeams.loc[j,'FTA RATE'])
            list_opp_tov.append(df_allTeams.loc[j,'TOV%'])
            list_opp_oreb.append(df_allTeams.loc[j,'OREB%'])
            break

0 WAS WAS
1 PHI PHI
2 SAC SAC
3 LAL LAL
4 WAS WAS
5 HOU HOU
6 CLE CLE
7 CHI CHI
8 PHI PHI
9 MIA MIA
10 MIL MIL
11 CHA CHA
12 NYK NYK
13 NOP NOP
14 IND IND
15 UTA UTA
16 LAL LAL
17 GSW GSW
18 PHX PHX
19 DET DET
20 TOR TOR
21 OKC OKC
22 MIA MIA
23 MIL MIL
24 ORL ORL
25 TOR TOR
26 CHA CHA
27 OKC OKC
28 MIN MIN
29 DEN DEN
30 MIN MIN
31 NYK NYK
32 DET DET
33 SAS SAS
34 ORL ORL
35 NOP NOP
36 DAL DAL
37 BKN BKN
38 BOS BOS
39 MIL MIL
40 NYK NYK
41 DET DET
42 CHI CHI
43 PHI PHI
44 LAC LAC
45 CHI CHI
46 WAS WAS
47 NYK NYK
48 MIA MIA
49 HOU HOU
50 ORL ORL
51 UTA UTA
52 DEN DEN
53 SAC SAC
54 POR POR
55 LAC LAC
56 MIA MIA
57 ORL ORL
58 BOS BOS
59 DAL DAL
60 CLE CLE
61 IND IND
62 GSW GSW
63 BKN BKN
64 TOR TOR
65 MEM MEM
66 SAS SAS
67 MEM MEM
68 POR POR
69 CHA CHA
70 WAS WAS
71 MIL MIL
72 BKN BKN
73 PHX PHX
74 PHI PHI
75 CHI CHI
76 BKN BKN
77 BOS BOS
78 CLE CLE
79 CLE CLE
80 CHA CHA
81 IND IND
82 BOS BOS
83 IND IND
84 MIL MIL
85 CHI CHI
86 DET DET
87 CHA CHA
88 MIN MIN
89 NYK NYK
90 PHX PHX
91 LAC LA

694 POR POR
695 SAC SAC
696 GSW GSW
697 UTA UTA
698 LAL LAL
699 ATL ATL
700 WAS WAS
701 SAC SAC
702 MIA MIA
703 BOS BOS
704 NOP NOP
705 MIN MIN
706 IND IND
707 PHI PHI
708 LAL LAL
709 SAS SAS
710 TOR TOR
711 MIL MIL
712 DAL DAL
713 CHA CHA
714 BOS BOS
715 POR POR
716 NOP NOP
717 PHI PHI
718 CHI CHI
719 IND IND
720 CLE CLE
721 NYK NYK
722 CLE CLE
723 UTA UTA
724 TOR TOR
725 PHX PHX
726 BKN BKN
727 CHI CHI
728 ORL ORL
729 NYK NYK
730 MIA MIA
731 BKN BKN
732 MIL MIL
733 TOR TOR
734 HOU HOU
735 MEM MEM
736 WAS WAS
737 ORL ORL
738 SAS SAS
739 NOP NOP
740 PHX PHX
741 POR POR
742 OKC OKC
743 LAL LAL
744 NOP NOP
745 DAL DAL
746 DEN DEN
747 PHX PHX
748 TOR TOR
749 BOS BOS
750 MIL MIL
751 IND IND
752 LAL LAL
753 LAL LAL
754 MIN MIN
755 ATL ATL
756 HOU HOU
757 PHX PHX
758 IND IND
759 LAC LAC
760 UTA UTA
761 MEM MEM
762 MIN MIN
763 NOP NOP
764 NYK NYK
765 POR POR
766 UTA UTA
767 BKN BKN
768 DET DET
769 CLE CLE
770 TOR TOR
771 DAL DAL
772 DEN DEN
773 POR POR
774 MEM MEM
775 SAC SAC
776 MIA MIA
777 

1352 HOU HOU
1353 ORL ORL
1354 MIA MIA
1355 HOU HOU
1356 PHI PHI
1357 TOR TOR
1358 BOS BOS
1359 UTA UTA
1360 DEN DEN
1361 PHX PHX
1362 MIA MIA
1363 LAL LAL
1364 IND IND
1365 DET DET
1366 BKN BKN
1367 UTA UTA
1368 PHX PHX
1369 CLE CLE
1370 DEN DEN
1371 LAC LAC
1372 TOR TOR
1373 PHI PHI
1374 NYK NYK
1375 IND IND
1376 MIN MIN
1377 MEM MEM
1378 LAC LAC
1379 LAL LAL
1380 GSW GSW
1381 POR POR
1382 SAC SAC
1383 ATL ATL
1384 CHI CHI
1385 CHA CHA
1386 BOS BOS
1387 DET DET
1388 DAL DAL
1389 OKC OKC
1390 IND IND
1391 PHI PHI
1392 CHA CHA
1393 BOS BOS
1394 MEM MEM
1395 SAC SAC
1396 MEM MEM
1397 DEN DEN
1398 OKC OKC
1399 BKN BKN
1400 ORL ORL
1401 LAC LAC
1402 LAL LAL
1403 CHA CHA
1404 PHI PHI
1405 MEM MEM
1406 BOS BOS
1407 NOP NOP
1408 PHX PHX
1409 GSW GSW
1410 UTA UTA
1411 NYK NYK
1412 NYK NYK
1413 CHA CHA
1414 SAS SAS
1415 TOR TOR
1416 DET DET
1417 GSW GSW
1418 CHI CHI
1419 HOU HOU
1420 PHX PHX
1421 ATL ATL
1422 SAC SAC
1423 OKC OKC
1424 ATL ATL
1425 DEN DEN
1426 MIL MIL
1427 POR POR
1428 PHI PHI

1987 IND IND
1988 MIA MIA
1989 CHI CHI
1990 MIL MIL
1991 MEM MEM
1992 IND IND
1993 LAC LAC
1994 OKC OKC
1995 DEN DEN
1996 GSW GSW
1997 SAC SAC
1998 DAL DAL
1999 SAS SAS
2000 TOR TOR
2001 SAC SAC
2002 SAS SAS
2003 MIN MIN
2004 GSW GSW
2005 LAL LAL
2006 DET DET
2007 LAL LAL
2008 CLE CLE
2009 ORL ORL
2010 WAS WAS
2011 CHA CHA
2012 PHI PHI
2013 BOS BOS
2014 LAL LAL
2015 MEM MEM
2016 GSW GSW
2017 CHA CHA
2018 DAL DAL
2019 OKC OKC
2020 DAL DAL
2021 BOS BOS
2022 ATL ATL
2023 UTA UTA
2024 ORL ORL
2025 TOR TOR
2026 DET DET
2027 OKC OKC
2028 BKN BKN
2029 OKC OKC
2030 PHI PHI
2031 WAS WAS
2032 PHX PHX
2033 NOP NOP
2034 SAS SAS
2035 ATL ATL
2036 MIA MIA
2037 MIL MIL
2038 NYK NYK
2039 MIN MIN
2040 LAL LAL
2041 DEN DEN
2042 HOU HOU
2043 PHX PHX
2044 MIN MIN
2045 UTA UTA
2046 MIN MIN
2047 UTA UTA
2048 SAS SAS
2049 NOP NOP
2050 PHX PHX
2051 SAS SAS
2052 MIN MIN
2053 ATL ATL
2054 MIA MIA
2055 ORL ORL
2056 MIL MIL
2057 TOR TOR
2058 NOP NOP
2059 LAL LAL
2060 POR POR
2061 SAS SAS
2062 LAC LAC
2063 TOR TOR

In [133]:
for i in range(2460):
    df_allTeams.loc[i,'OPP_EFG%'] = list_opp_efg[i]
    df_allTeams.loc[i,'OPP_FTA RATE'] = list_opp_fta[i]
    df_allTeams.loc[i,'OPP_TOV%'] = list_opp_tov[i]
    df_allTeams.loc[i,'OPP_OREB%'] = list_opp_oreb[i]
    

df_allTeams.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.5448,0.2716,0.1554,0.2328,0.5578,0.2432,0.1564,0.2244,WAS,1,0,0,1
1,ATL,10/29/16,48,0.5472,0.2362,0.1648,0.2360,0.4718,0.2934,0.1534,0.2260,PHI,0,2,3,1
2,ATL,10/31/16,48,0.5328,0.2724,0.1660,0.2360,0.5198,0.3660,0.1358,0.2142,SAC,1,2,2,1
3,ATL,11/2/16,48,0.5270,0.3126,0.1718,0.2654,0.4792,0.2410,0.1762,0.2460,LAL,1,2,1,0
4,ATL,11/4/16,48,0.5238,0.3352,0.1692,0.2530,0.5206,0.2338,0.1624,0.2026,WAS,0,2,2,0


In [142]:
df_save_team=df_allTeams

In [126]:
df_all.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,POR,10/25/16,48,0.607,0.293,0.140,0.167,0.537,0.195,0.144,0.171,UTA,1,0,0,1
1,UTA,10/25/16,48,0.537,0.195,0.144,0.171,0.607,0.293,0.140,0.167,POR,0,0,0,0
2,NYK,10/25/16,48,0.420,0.230,0.179,0.245,0.548,0.202,0.141,0.275,CLE,0,0,0,0
3,GSW,10/25/16,48,0.512,0.212,0.159,0.190,0.541,0.265,0.137,0.438,SAS,1,0,0,0
4,SAS,10/25/16,48,0.541,0.265,0.137,0.438,0.512,0.212,0.159,0.190,GSW,0,0,0,1


In [ ]:
# list_teams = ['ATL','BKN','BOS','CHA','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM','MIA',
#               'MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHX','POR','SAC','SAS','TOR','UTA','WAS']
# df_allTeams = pd.DataFrame()    
# for i in list_teams:
#     df_allTeams = pd.concat([df_allTeams, df_test.loc[df['TEAM'] == i]])
# df_allTeams.head()

In [156]:
df_reorder = pd.DataFrame()
for i in range(2460):
    for j in range(2460):
        if df_allTeams.loc[j,'TEAM']==df_all.loc[i,'TEAM'] and df_allTeams.loc[j,'GAME DATE']==df_all.loc[i,'GAME DATE']:
#             print(df_allTeams.iloc[j,:])
            df_reorder = pd.concat([df_reorder, df_allTeams.loc[j,:]], axis=1)
            break


# df_reorder.head()

In [158]:
df_re=df_reorder.T
df_re.head()

,EFG%,FTA RATE,GAME DATE,Home/Away,MIN,OPP,OPP_EFG%,OPP_FTA RATE,OPP_OREB%,OPP_REST,OPP_TOV%,OREB%,REST,TEAM,TOV%,W/L
1968,0.4946,0.2048,10/25/16,1,48,UTA,0.5662,0.2562,0.2276,0,0.1516,0.2112,0,POR,0.1204,1
2296,0.5662,0.2562,10/25/16,0,48,POR,0.4946,0.2048,0.2112,0,0.1204,0.2276,0,UTA,0.1516,0
1558,0.4968,0.2236,10/25/16,0,48,CLE,0.5254,0.2346,0.2052,0,0.1494,0.3372,0,NYK,0.145,0
738,0.563,0.236,10/25/16,1,48,SAS,0.4912,0.1726,0.25,0,0.141,0.2434,0,GSW,0.14,0
2132,0.4912,0.1726,10/25/16,0,48,GSW,0.563,0.236,0.2434,0,0.14,0.25,0,SAS,0.141,1


In [159]:
df_all.head(10)

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,POR,10/25/16,48,0.607,0.293,0.140,0.167,0.537,0.195,0.144,0.171,UTA,1,0,0,1
1,UTA,10/25/16,48,0.537,0.195,0.144,0.171,0.607,0.293,0.140,0.167,POR,0,0,0,0
2,NYK,10/25/16,48,0.420,0.230,0.179,0.245,0.548,0.202,0.141,0.275,CLE,0,0,0,0
3,GSW,10/25/16,48,0.512,0.212,0.159,0.190,0.541,0.265,0.137,0.438,SAS,1,0,0,0
4,SAS,10/25/16,48,0.541,0.265,0.137,0.438,0.512,0.212,0.159,0.190,GSW,0,0,0,1
5,CLE,10/25/16,48,0.548,0.202,0.141,0.275,0.420,0.230,0.179,0.245,NYK,1,0,0,1
6,NOP,10/26/16,48,0.429,0.283,0.101,0.096,0.494,0.398,0.224,0.282,DEN,1,0,0,0
7,PHX,10/26/16,48,0.494,0.351,0.186,0.244,0.547,0.302,0.138,0.244,SAC,1,0,0,0
8,SAC,10/26/16,48,0.547,0.302,0.138,0.244,0.494,0.351,0.186,0.244,PHX,0,0,0,1
9,HOU,10/26/16,48,0.552,0.314,0.146,0.263,0.573,0.247,0.154,0.275,LAL,0,0,0,0


In [162]:
df_re=df_re[['TEAM','GAME DATE','MIN','EFG%','FTA RATE','TOV%','OREB%','OPP_EFG%','OPP_FTA RATE','OPP_TOV%','OPP_OREB%','OPP','Home/Away','REST','OPP_REST','W/L']]
df_re.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
1968,POR,10/25/16,48,0.4946,0.2048,0.1204,0.2112,0.5662,0.2562,0.1516,0.2276,UTA,1,0,0,1
2296,UTA,10/25/16,48,0.5662,0.2562,0.1516,0.2276,0.4946,0.2048,0.1204,0.2112,POR,0,0,0,0
1558,NYK,10/25/16,48,0.4968,0.2236,0.145,0.3372,0.5254,0.2346,0.1494,0.2052,CLE,0,0,0,0
738,GSW,10/25/16,48,0.563,0.236,0.14,0.2434,0.4912,0.1726,0.141,0.25,SAS,1,0,0,0
2132,SAS,10/25/16,48,0.4912,0.1726,0.141,0.25,0.563,0.236,0.14,0.2434,GSW,0,0,0,1


In [163]:
df_re.to_csv('AllGames_Test.csv', index = False)
df_test = pd.read_csv('AllGames_Test.csv', encoding = 'latin1')
df_test.to_csv('AllGames_Test.csv')

In [148]:
df_allTeams.iloc[:,1]

0       10/27/16
1       10/29/16
2       10/31/16
3        11/2/16
4        11/4/16
5        11/5/16
6        11/8/16
7        11/9/16
8       11/12/16
9       11/15/16
10      11/16/16
11      11/18/16
12      11/20/16
13      11/22/16
14      11/23/16
15      11/25/16
16      11/27/16
17      11/28/16
18      11/30/16
19       12/2/16
20       12/3/16
21       12/5/16
22       12/7/16
23       12/9/16
24      12/13/16
25      12/16/16
26      12/17/16
27      12/19/16
28      12/21/16
29      12/23/16
          ...   
2430     2/10/17
2431     2/13/17
2432     2/16/17
2433     2/24/17
2434     2/26/17
2435     2/28/17
2436      3/1/17
2437      3/3/17
2438      3/5/17
2439      3/7/17
2440      3/8/17
2441     3/10/17
2442     3/11/17
2443     3/13/17
2444     3/15/17
2445     3/17/17
2446     3/18/17
2447     3/20/17
2448     3/22/17
2449     3/24/17
2450     3/25/17
2451     3/28/17
2452     3/29/17
2453     3/31/17
2454      4/2/17
2455      4/4/17
2456      4/6/17
2457      4/8/

In [44]:
# df_schedule=df
# df_schedule.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,POR,10/25/16,48,0.607,0.293,0.140,0.167,0.537,0.195,0.144,0.171,UTA,1,0,0,1
1,UTA,10/25/16,48,0.537,0.195,0.144,0.171,0.607,0.293,0.140,0.167,POR,0,0,0,0
2,NYK,10/25/16,48,0.420,0.230,0.179,0.245,0.548,0.202,0.141,0.275,CLE,0,0,0,0
3,GSW,10/25/16,48,0.512,0.212,0.159,0.190,0.541,0.265,0.137,0.438,SAS,1,0,0,0
4,SAS,10/25/16,48,0.541,0.265,0.137,0.438,0.512,0.212,0.159,0.190,GSW,0,0,0,1


In [46]:
# df_schedule=df_schedule[['TEAM','GAME DATE','OPP','Home/Away','REST','OPP_REST']]
# df_schedule.head()

,TEAM,GAME DATE,OPP,Home/Away,REST,OPP_REST
0,POR,10/25/16,UTA,1,0,0
1,UTA,10/25/16,POR,0,0,0
2,NYK,10/25/16,CLE,0,0,0
3,GSW,10/25/16,SAS,1,0,0
4,SAS,10/25/16,GSW,0,0,0


In [47]:
# df_schedule.to_csv('1617_Schedule.csv')

### IGNORE train set

In [71]:
df_train=df.iloc[0:2214]
len(df_train)

2214

In [72]:
df_test=df.iloc[2214:2460]
len(df_test)

246

In [75]:
df_test.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
2214,POR,3/28/17,48,0.538,0.363,0.090,0.300,0.516,0.213,0.125,0.273,DEN,1,2,2,1
2215,IND,3/28/17,48,0.525,0.432,0.149,0.314,0.532,0.474,0.144,0.297,MIN,1,2,3,0
2216,BKN,3/28/17,48,0.452,0.255,0.124,0.241,0.484,0.231,0.153,0.245,PHI,1,2,2,0
2217,ATL,3/28/17,48,0.435,0.341,0.176,0.209,0.401,0.250,0.139,0.250,PHX,1,2,2,1
2218,LAC,3/29/17,48,0.639,0.446,0.099,0.216,0.565,0.312,0.107,0.298,WAS,1,3,1,1


In [ ]:
# for i in range(len(df_test)):
#     print('------',i,df_test['TEAM'][i],'------')
#     df_test['MIN'][i]=48
#     count=0
#     opp_count=0
#     #a=0
#     #b=0
#     efg=0
#     fta=0
#     tov=0
#     oreb=0
#     opp_efg=0
#     opp_fta=0
#     opp_tov=0
#     opp_oreb=0
#     for j in range(len(df_train)):
#         if df_train['TEAM'][j]==df_test['TEAM'][i] and df_train['Home/Away'][j]==df_test['Home/Away'][i]:
#             #a=1
#             count += 1
#             efg += df_train['EFG%'][j]
#             fta += df_train['FTA RATE'][j]
#             tov += df_train['TOV%'][j]
#             oreb += df_train['OREB%'][j]
#         if df_train['OPP'][j]==df_test['OPP'][i] and df_train['Home/Away'][j]==df_test['Home/Away'][i]:
#             #b=1
#             opp_count += 1
#             opp_efg += df_train['OPP_EFG%'][j]
#             opp_fta += df_train['OPP_FTA RATE'][j]
#             opp_tov += df_train['OPP_TOV%'][j]
#             opp_oreb += df_train['OPP_OREB%'][j]
# #         if a==1 and b==1:
# #             break
#     df_test['EFG%'][i]=(efg/count)
#     df_test['FTA RATE'][i]=(fta/count)
#     df_test['TOV%'][i]=(tov/count)
#     df_test['OREB%'][i]=(oreb/count)
#     df_test['OPP_EFG%'][i]=(opp_efg/opp_count)
#     df_test['OPP_FTA RATE'][i]=(opp_fta/opp_count)
#     df_test['OPP_TOV%'][i]=(opp_tov/opp_count)
#     df_test['OPP_OREB%'][i]=(opp_oreb/opp_count)
    
            
            
            
        
    

### test set

In [46]:
df_allTeams = pd.read_csv('data/AllGames_byTeam.csv', encoding = 'latin1',index_col=0)
df_allTeams.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.568,0.205,0.204,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
1,ATL,10/29/16,48,0.524,0.271,0.119,0.146,0.381,0.188,0.195,0.089,PHI,0,2,3,1
2,ATL,10/31/16,48,0.487,0.595,0.136,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
3,ATL,11/2/16,48,0.547,0.341,0.170,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
4,ATL,11/4/16,48,0.416,0.434,0.168,0.222,0.419,0.438,0.184,0.229,WAS,0,2,2,0


In [47]:
df_home = df_allTeams[df_allTeams['Home/Away'] != 0]
df_home.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.568,0.205,0.204,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
2,ATL,10/31/16,48,0.487,0.595,0.136,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
3,ATL,11/2/16,48,0.547,0.341,0.170,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
5,ATL,11/5/16,48,0.576,0.282,0.195,0.295,0.506,0.153,0.248,0.279,HOU,1,1,3,1
7,ATL,11/9/16,48,0.560,0.325,0.219,0.361,0.500,0.231,0.132,0.186,CHI,1,1,2,1


In [10]:
len(df_home)

1230

In [11]:
efg=pd.expanding_mean(df_home['EFG%'])

/Users/user/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pd.expanding_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.expanding(min_periods=1).mean()
  """Entry point for launching an IPython kernel.


In [17]:
list_teams = ['ATL','BKN','BOS','CHA','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM','MIA',
              'MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHX','POR','SAC','SAS','TOR','UTA','WAS']

In [28]:
list_efg=[]
list_fta=[]
list_tov=[]
list_oreb=[]
for item in list_teams:
    df_team = df_home[df_home['TEAM'] == item]
    list_efg.extend((pd.expanding_mean(df_team['EFG%'])).tolist())
    list_fta.extend((pd.expanding_mean(df_team['FTA RATE'])).tolist())
    list_tov.extend((pd.expanding_mean(df_team['TOV%'])).tolist())
    list_oreb.extend((pd.expanding_mean(df_team['OREB%'])).tolist())

/Users/user/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.expanding_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.expanding(min_periods=1).mean()
  import sys
/Users/user/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pd.expanding_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.expanding(min_periods=1).mean()
  
/Users/user/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pd.expanding_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.expanding(min_periods=1).mean()
  if __name__ == '__main__':
/Users/user/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: pd.expanding_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.expanding(min_periods=1).mean()
  # Remove the CWD from sys.path while we load

In [29]:
len(list_efg)

1230

In [26]:
df_team = df_home[df_home['TEAM'] == 'ATL']

In [27]:
df_team.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.568,0.205,0.204,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
2,ATL,10/31/16,48,0.487,0.595,0.136,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
3,ATL,11/2/16,48,0.547,0.341,0.170,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
5,ATL,11/5/16,48,0.576,0.282,0.195,0.295,0.506,0.153,0.248,0.279,HOU,1,1,3,1
7,ATL,11/9/16,48,0.560,0.325,0.219,0.361,0.500,0.231,0.132,0.186,CHI,1,1,2,1


In [40]:
df_save_home=df_home
df_save_home.head()

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48,0.568,0.205,0.204,0.333,0.457,0.191,0.174,0.240,WAS,1,0,0,1
2,ATL,10/31/16,48,0.487,0.595,0.136,0.347,0.547,0.253,0.197,0.188,SAC,1,2,2,1
3,ATL,11/2/16,48,0.547,0.341,0.170,0.244,0.583,0.222,0.146,0.289,LAL,1,2,1,0
5,ATL,11/5/16,48,0.576,0.282,0.195,0.295,0.506,0.153,0.248,0.279,HOU,1,1,3,1
7,ATL,11/9/16,48,0.560,0.325,0.219,0.361,0.500,0.231,0.132,0.186,CHI,1,1,2,1


In [44]:
for i in range(1230):
    df_save_home.loc[i,'EFG%'] = list_efg[i]
    df_save_home.loc[i,'FTA RATE'] = list_fta[i]
    df_save_home.loc[i,'TOV%'] = list_tov[i]
    df_save_home.loc[i,'OREB%'] = list_oreb[i]

df_save_home.head()    

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
0,ATL,10/27/16,48.0,0.56800,0.205000,0.204000,0.333000,0.457,0.191,0.174,0.240,WAS,1.0,0.0,0.0,1.0
2,ATL,10/31/16,48.0,0.53400,0.380333,0.170000,0.308000,0.547,0.253,0.197,0.188,SAC,1.0,2.0,2.0,1.0
3,ATL,11/2/16,48.0,0.54450,0.355750,0.176250,0.304750,0.583,0.222,0.146,0.289,LAL,1.0,2.0,1.0,0.0
5,ATL,11/5/16,48.0,0.55200,0.315833,0.180167,0.320167,0.506,0.153,0.248,0.279,HOU,1.0,1.0,3.0,1.0
7,ATL,11/9/16,48.0,0.54825,0.305500,0.174375,0.269125,0.500,0.231,0.132,0.186,CHI,1.0,1.0,2.0,1.0


In [ ]:
len()

In [53]:
df_4 = df_home.groupby('TEAM')['EFG%', 'FTA RATE', 'TOV%', 'OREB%'].expanding().mean()

In [59]:
df_4['TEAM'] = list(df_home['TEAM'].values)

In [61]:
df_4.head(-5)

EFG%  FTA RATE      TOV%     OREB% TEAM
TEAM                                                  
ATL  0     0.568000  0.205000  0.204000  0.333000  ATL
     2     0.527500  0.400000  0.170000  0.340000  ATL
     3     0.534000  0.380333  0.170000  0.308000  ATL
     5     0.544500  0.355750  0.176250  0.304750  ATL
     7     0.547600  0.349600  0.184800  0.316000  ATL
     8     0.552000  0.315833  0.180167  0.320167  ATL
     10    0.552571  0.332286  0.177571  0.286714  ATL
     13    0.548250  0.305500  0.174375  0.269125  ATL
     19    0.531556  0.296778  0.163000  0.252778  ATL
     21    0.527100  0.308700  0.164300  0.244200  ATL
     22    0.525636  0.296091  0.162818  0.254636  ATL
     24    0.529083  0.291083  0.161750  0.263500  ATL
     26    0.528615  0.283000  0.158923  0.258615  ATL
     28    0.523500  0.274143  0.155500  0.252857  ATL
     31    0.515333  0.273667  0.154133  0.255533  ATL
     32    0.513687  0.274312  0.153688  0.257875  ATL
     33    0.514824  0.275412  0.151353  0.255882  ATL
     38    0.513667  0.274000  0.150111  0.256167  ATL
     39    0.518526  0.271421  0.152947  0.262579  ATL
     42    0.519100  0.269450  0.155250  0.262250  ATL
     43    0.519762  0.268667  0.154238  0.261952  ATL
     44    0.517591  0.269864  0.150864  0.258318  ATL
     46    0.513174  0.271217  0.152522  0.254478  ATL
     47    0.511750  0.273917  0.150542  0.254792  ATL
     50    0.514880  0.276800  0.151520  0.255720  ATL
     51    0.513769  0.282769  0.150654  0.252308  ATL
     52    0.515037  0.279481  0.149148  0.254074  ATL
     56    0.511571  0.285786  0.147500  0.251071  ATL
     59    0.516034  0.288862  0.150897  0.247517  ATL
     60    0.518600  0.290233  0.149667  0.248667  ATL
...             ...       ...       ...       ...  ...
WAS  2389  0.524143  0.269714  0.159714  0.276286  WAS
     2390  0.516375  0.282750  0.154125  0.274750  WAS
     2392  0.515778  0.269000  0.153222  0.274556  WAS
     2393  0.511300  0.272700  0.159800  0.273300  WAS
     2397  0.513182  0.271364  0.155636  0.268636  WAS
     2398  0.511000  0.266917  0.159083  0.263333  WAS
     2399  0.510154  0.270846  0.154692  0.263615  WAS
     2401  0.512714  0.265429  0.154000  0.266214  WAS
     2402  0.521800  0.271133  0.148067  0.254533  WAS
     2403  0.526250  0.277313  0.148187  0.253812  WAS
     2407  0.529529  0.275882  0.149706  0.250000  WAS
     2408  0.530000  0.286889  0.151667  0.255056  WAS
     2409  0.535421  0.280684  0.154105  0.254000  WAS
     2412  0.541550  0.272350  0.153550  0.249350  WAS
     2414  0.543000  0.264524  0.156143  0.246000  WAS
     2416  0.542273  0.261500  0.154136  0.246682  WAS
     2417  0.543217  0.262783  0.152826  0.248087  WAS
     2418  0.541417  0.265375  0.151917  0.250250  WAS
     2422  0.544760  0.264760  0.152160  0.250240  WAS
     2425  0.546769  0.261115  0.151038  0.249731  WAS
     2426  0.545815  0.259556  0.149667  0.249926  WAS
     2427  0.544821  0.259750  0.149036  0.247250  WAS
     2428  0.545552  0.259862  0.147621  0.247138  WAS
     2430  0.544800  0.262567  0.146433  0.249033  WAS
     2431  0.548645  0.266484  0.148968  0.246871  WAS
     2434  0.546219  0.262813  0.149187  0.243156  WAS
     2435  0.545303  0.261515  0.150576  0.247242  WAS
     2437  0.541735  0.264059  0.149324  0.248000  WAS
     2438  0.542486  0.265343  0.148714  0.248057  WAS
     2444  0.543000  0.265083  0.147972  0.244639  WAS

[1225 rows x 5 columns]

In [45]:
df_save_home.iloc[40:45,:]

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
80,ATL,4/11/17,48.0,0.500450,0.284275,0.162825,0.205825,0.347,0.216,0.156,0.278,CHA,1.0,2.0,1.0,1.0
83,BKN,10/28/16,48.0,0.576500,0.198500,0.153500,0.260000,0.413,0.204,0.123,0.259,IND,1.0,2.0,2.0,1.0
85,BKN,10/31/16,48.0,0.525750,0.277000,0.144500,0.272250,0.567,0.202,0.192,0.310,CHI,1.0,2.0,2.0,0.0
86,BKN,11/2/16,48.0,0.516600,0.282200,0.156800,0.282800,0.472,0.239,0.130,0.200,DET,1.0,2.0,1.0,1.0
87,BKN,11/4/16,48.0,0.503833,0.273333,0.153000,0.260000,0.477,0.279,0.152,0.152,CHA,1.0,2.0,2.0,0.0


In [48]:
df_home.iloc[40:45,:]

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
80,ATL,4/11/17,48,0.563,0.228,0.183,0.306,0.347,0.216,0.156,0.278,CHA,1,2,1,1
83,BKN,10/28/16,48,0.494,0.230,0.160,0.255,0.413,0.204,0.123,0.259,IND,1,2,2,1
85,BKN,10/31/16,48,0.370,0.219,0.144,0.262,0.567,0.202,0.192,0.310,CHI,1,2,2,0
86,BKN,11/2/16,48,0.625,0.224,0.195,0.152,0.472,0.239,0.130,0.200,DET,1,2,1,1
87,BKN,11/4/16,48,0.454,0.241,0.152,0.143,0.477,0.279,0.152,0.152,CHA,1,2,2,0


In [33]:
df_save_home.iloc[40:45,:]

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
80,ATL,4/11/17,48.0,0.500450,0.284275,0.162825,0.205825,0.347,0.216,0.156,0.278,CHA,1.0,2.0,1.0,1.0
83,BKN,10/28/16,48.0,0.576500,0.198500,0.153500,0.260000,0.413,0.204,0.123,0.259,IND,1.0,2.0,2.0,1.0
85,BKN,10/31/16,48.0,0.525750,0.277000,0.144500,0.272250,0.567,0.202,0.192,0.310,CHI,1.0,2.0,2.0,0.0
86,BKN,11/2/16,48.0,0.516600,0.282200,0.156800,0.282800,0.472,0.239,0.130,0.200,DET,1.0,2.0,1.0,1.0
87,BKN,11/4/16,48.0,0.503833,0.273333,0.153000,0.260000,0.477,0.279,0.152,0.152,CHA,1.0,2.0,2.0,0.0


In [32]:
df_home.iloc[40:45,:]

,TEAM,GAME DATE,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP_EFG%,OPP_FTA RATE,OPP_TOV%,OPP_OREB%,OPP,Home/Away,REST,OPP_REST,W/L
80,ATL,4/11/17,48.0,0.500450,0.284275,0.162825,0.205825,0.347,0.216,0.156,0.278,CHA,1.0,2.0,1.0,1.0
83,BKN,10/28/16,48.0,0.576500,0.198500,0.153500,0.260000,0.413,0.204,0.123,0.259,IND,1.0,2.0,2.0,1.0
85,BKN,10/31/16,48.0,0.525750,0.277000,0.144500,0.272250,0.567,0.202,0.192,0.310,CHI,1.0,2.0,2.0,0.0
86,BKN,11/2/16,48.0,0.516600,0.282200,0.156800,0.282800,0.472,0.239,0.130,0.200,DET,1.0,2.0,1.0,1.0
87,BKN,11/4/16,48.0,0.503833,0.273333,0.153000,0.260000,0.477,0.279,0.152,0.152,CHA,1.0,2.0,2.0,0.0


In [ ]:
list_opp_efg=[]
list_opp_fta=[]
list_opp_tov=[]
list_opp_oreb=[]
for i in range(1230):
    for j in range(1230):
        if df_allTeams.loc[j,'TEAM']==df_allTeams.loc[i,'OPP'] and df_allTeams.loc[j,'GAME DATE']==df_allTeams.loc[i,'GAME DATE']:
            print(i,df_allTeams.loc[i,'OPP'],df_allTeams.loc[j,'TEAM'])
            list_opp_efg.append(df_allTeams.loc[j,'EFG%'])
            list_opp_fta.append(df_allTeams.loc[j,'FTA RATE'])
            list_opp_tov.append(df_allTeams.loc[j,'TOV%'])
            list_opp_oreb.append(df_allTeams.loc[j,'OREB%'])
            break